In [44]:
import numpy as np
import pandas as pd

from scipy import stats
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.weightstats import CompareMeans, DescrStatsW, ztest

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

import warnings
warnings.filterwarnings('ignore')

Прежде всего, скопируем нужные функции из учебного ноутбука. Они понадобятся.

In [52]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)
    
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. 

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции $\textbf{sklearn.cross_validation.train_test_split}$ (зафиксируйте $\textbf{random state = 1)}$. На оставшихся $150$ настройте два классификатора поддельности банкнот:

1. логистическая регрессия по признакам $X_1,X_2,X_3$ 

2. логистическая регрессия по признакам $X_4,X_5,X_6$ 

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$,  нужно ввести 8).

In [18]:
df = pd.read_table('banknotes.txt')

y = df['real']
X = df.drop(['real'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 1, test_size = 50)

X1_train = X_train[['X1', 'X2','X3']]
X2_train = X_train[['X4','X5','X6']]
X1_test = X_test[['X1', 'X2','X3']]
X2_test = X_test[['X4','X5','X6']]

logreg = LogisticRegression()
logreg.fit(X1_train, y_train)
pred1 = logreg.predict(X1_test)
logreg.fit(X2_train, y_train)
pred2 = logreg.predict(X2_test)

pred1_acc = np.array([1 if pred1[i] == np.array(y_test)[i] else 0 for i in range(len(pred1))])
pred2_acc = np.array([1 if pred2[i] == np.array(y_test)[i] else 0 for i in range(len(pred2))])

print('First prediction accuracy:', sum(pred1_acc)/len(pred1_acc),
      '\n','Second prediction accuracy:', sum(pred2_acc)/len(pred2_acc))

Вывод - доли ошибок не одинаковы

В предыдущей задаче посчитайте $95\%$ доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.


Построим $95\%$ доверительный интервал для разницы предсказаний

In [50]:
print('95%% доверительный интервал для разницы предсказаний: [%.4f, %.4f]' %
      proportions_diff_confint_rel(pred1_acc, pred2_acc))

95% доверительный интервал для разницы предсказаний: [-0.3001, -0.0599]


In [53]:
print ("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(pred1_acc, pred2_acc)))

p-value: 0.003297


Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [59]:
n = 100
mean_result = 525
stand_dev = 100
mean_spec = 541.4
alpha = 0.05

Реализуем формулу: $Z(X^n) = \frac{\overline{X}-\mu_0}{\frac{\sigma}{\sqrt{n}}}$

In [60]:
def z_conf(mu, sigma, n, x_mean):
    return (x_mean - mu)/(sigma / np.sqrt(n)) 

In [61]:
print((z_conf(mu = mean_result, x_mean=mean_spec, n=n, sigma=stand_dev)))

1.6399999999999977


In [66]:
print(round(1-stats.norm.cdf(z_conf(mu = mean_result, x_mean=mean_spec, n=n, sigma=stand_dev)),4))

0.0505


Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.



In [63]:
print((z_conf(mu = mean_result, x_mean=541.5, n=n, sigma=stand_dev)))

1.65


In [65]:
print(round(1-stats.norm.cdf(z_conf(mu = mean_result, x_mean=541.5, n=n, sigma=stand_dev)),4))

0.0495
